<a href="https://colab.research.google.com/github/psantul/Mtech_thesis_2022/blob/main/DecisionTree_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 48.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6d1568a1b041202f2941a70f2819f3adc9deeabc4b81c7c769562c24e8088982
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
pip install findspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('RF_project').getOrCreate()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

dataset = spark.read.csv('/content/drive/MyDrive/LALR.csv',inferSchema=True, header =True)

In [ ]:
dataset = dataset.filter(dataset.modelMag_u > 0)
dataset = dataset.filter(dataset.modelMag_g > 0)
dataset = dataset.filter(dataset.modelMag_r > 0)
dataset = dataset.filter(dataset.modelMag_i > 0)
dataset = dataset.filter(dataset.modelMag_z > 0)
dataset = dataset.filter(dataset.modelMag_ug > 0)
dataset = dataset.filter(dataset.modelMag_gr > 0)
dataset = dataset.filter(dataset.modelMag_ri > 0)
dataset = dataset.filter(dataset.modelMag_iz > 0)
dataset = dataset.filter(dataset.fiberMag_u > 0)
dataset = dataset.filter(dataset.fiberMag_g > 0)
dataset = dataset.filter(dataset.fiberMag_r > 0)
dataset = dataset.filter(dataset.fiberMag_i > 0)
dataset = dataset.filter(dataset.fiberMag_z > 0)
dataset = dataset.filter(dataset.fiberMag_ug > 0)
dataset = dataset.filter(dataset.fiberMag_gr > 0)
dataset = dataset.filter(dataset.fiberMag_ri > 0)
dataset = dataset.filter(dataset.fiberMag_iz > 0)
dataset = dataset.filter(dataset.petroR50_rr > 0)
dataset = dataset.filter(dataset.petroR90_zz > 0)
dataset = dataset.filter(dataset.ri > 0)
dataset = dataset.filter(dataset.iz > 0)
dataset = dataset.filter(dataset.dered_u > 0)
dataset = dataset.filter(dataset.dered_g > 0)
dataset = dataset.filter(dataset.dered_r > 0)
dataset = dataset.filter(dataset.dered_i > 0)
dataset = dataset.filter(dataset.dered_z > 0)
dataset = dataset.filter(dataset.petroMag_uu > 0)
dataset = dataset.filter(dataset.petroMag_gg > 0)
dataset = dataset.filter(dataset.petroMag_rr > 0)
dataset = dataset.filter(dataset.petroMag_ii > 0)
dataset = dataset.filter(dataset.petroMag_zz > 0)
dataset = dataset.filter(dataset.redshift > 0)

In [ ]:
import pyspark.sql.functions as func
dataset = dataset.withColumn("redshift", func.round(dataset["redshift"], 2).cast('integer'))

In [ ]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [ ]:
cols=new_data.columns
# cols = ['modelMag_u', 'modelMag_g', 'modelMag_r']
# cols.remove("Classes")
cols.remove("redshift")
cols.remove("petroR90_zz")
cols.remove("ri")
cols.remove("iz")


assembler = VectorAssembler(inputCols=cols,outputCol="features")

# Now let us use the transform method to transform our dataset
assembled_data=assembler.transform(new_data)

assembled_data.select("features",'redshift').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_Features")
# rescale each feature to range [min, max].
scaled_data = scaler.fit(assembled_data).transform(assembled_data)

In [ ]:
scaled_data.select("features",'redshift','Scaled_features').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
scaled_data = scaled_data.select("Scaled_features","redshift")
scaled_data.show()

+--------------------+--------+
|     Scaled_features|redshift|
+--------------------+--------+
|[0.34063284805936...|     1.0|
|[0.36747337681911...|     0.0|
|[0.35872741769952...|     0.0|
|[0.27374705083133...|     0.0|
|[0.37969480247973...|     1.0|
|[0.69769927275202...|     0.0|
|[0.35506255906072...|     1.0|
|[0.35144013132364...|     0.0|
|[0.66373642984624...|     0.0|
|[0.50716684449339...|     0.0|
|[0.67467001134142...|     0.0|
|[0.70975550518850...|     0.0|
|[0.61553094759695...|     0.0|
|[0.58377418008912...|     0.0|
|[0.55988812385580...|     0.0|
|[0.46091307322601...|     0.0|
|[0.40211699260376...|     0.0|
|[0.43791331455176...|     0.0|
|[0.75895579318824...|     0.0|
|[0.35757747396466...|     0.0|
+--------------------+--------+
only showing top 20 rows



In [ ]:
train_data, test_data = scaled_data.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(featuresCol = 'Scaled_features', labelCol = 'redshift')
dtModel = dt.fit(train_data)
predictions = dtModel.transform(test_data)
predictions.select('Scaled_features', 'redshift', 'rawPrediction', 'probability', 'prediction').show(25)


+--------------------+--------+--------------------+--------------------+----------+
|     Scaled_features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[0.05571928776195...|     0.0|[286422.0,2784.0,...|[0.98261683974352...|       0.0|
|[0.08798472885327...|     0.0|[286422.0,2784.0,...|[0.98261683974352...|       0.0|
|[0.11081294071784...|     0.0|[286422.0,2784.0,...|[0.98261683974352...|       0.0|
|[0.11438271656776...|     0.0|[286422.0,2784.0,...|[0.98261683974352...|       0.0|
|[0.12663718087321...|     0.0|[286422.0,2784.0,...|[0.98261683974352...|       0.0|
|[0.13024170807362...|     0.0|[286422.0,2784.0,...|[0.98261683974352...|       0.0|
|[0.13090386218443...|     0.0|[286422.0,2784.0,...|[0.98261683974352...|       0.0|
|[0.13237999347624...|     0.0|[286422.0,2784.0,...|[0.98261683974352...|       0.0|
|[0.13770689484473...|     0.0|[286422.0,2784.0,...|[0.9826168397

In [ ]:
predictions.select("redshift","prediction").show(10)

+--------+----------+
|redshift|prediction|
+--------+----------+
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
|     0.0|       0.0|
+--------+----------+
only showing top 10 rows



In [ ]:
predictionAndLabels = predictions.select("redshift","prediction").rdd

In [ ]:
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="redshift", predictionCol="prediction", metricName="accuracy")
accuracy_NB = evaluator.evaluate(predictions)

In [ ]:
print ("Accuracy",accuracy_NB)

Accuracy 0.9487259399621023


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="redshift", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(labelCol="redshift", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("Mean Absolute Eroor (MAE) on test data = %g" % mae)



Root Mean Squared Error (RMSE) on test data = 0.434844
Mean Absolute Eroor (MAE) on test data = 0.0841478


In [ ]:
from sklearn.metrics import confusion_matrix
y_pred=predictions.select("prediction").collect()
y_orig=predictions.select("redshift").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[73234   415   572    23     0     0     0     0]
 [ 1270  1276   302    25     0     0     0     0]
 [  721   222  1410    55     0     0     0     0]
 [  215     6    71   183     0     0     0     0]
 [   68     1     3     3     0     0     0     0]
 [   50     7     2     0     0     0     0     0]
 [   65     3     6     0     0     0     0     0]
 [    8     0     0     0     0     0     0     0]]
